# GBRT experimets - environment test

## install python packages, and julia binary

In [1]:
import os, sys

if sys.version_info[:2] != (3,9):
    print("WARNING: should run notebook with python 3.9")

print("\n ---install required python packages--- \n")
!pip install -r requirements.txt

print("\n ---download julia binary--- \n")
!wget https://julialang-s3.julialang.org/bin/linux/x64/1.9/julia-1.9.2-linux-x86_64.tar.gz

print("\n ---unzip julia binary--- \n")
!tar -xf julia-1.9.2-linux-x86_64.tar.gz

print("\n ---install PyJulia--- \n")
import julia
julia.install(julia = "julia-1.9.2/bin/julia")


 ---install required python packages--- 

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 68 kB 2.9 MB/s             
     |████████████████████████████████| 36.5 MB 63.9 MB/s            
     |████████████████████████████████| 18.3 MB 43.6 MB/s            
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 0.7.3 requires tensorboard>=1.14, which is not installed.
pytorch-lightning 0.7.3 requires torch>=1.1, which is not installed.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.25.2 which is incompatible.

 ---download julia binary--- 

--2023-08-14 08:40:33--  https://julialang-s3.julialang.org/bin/linux/x64/1.9/julia-1.9.2-linux-x86_64.tar.gz
Resolving julialang-s3.julialang.org (julialang-s3.julialang.org)... 151.101.66.49, 151.101.194.49, 151.101.2.49, ...
C

[ Info: Julia version info


Julia Version 1.9.2
Commit e4ee485e909 (2023-07-05 09:39 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  uname: Linux 5.15.0-78-generic #85~20.04.1-Ubuntu SMP Mon Jul 17 09:42:39 UTC 2023 x86_64 x86_64
  CPU: Intel(R) Xeon(R) Silver 4114 CPU @ 2.20GHz: 
                 speed         user         nice          sys         idle          irq
       #1-40   800 MHz    2082459 s   11222015 s    1307504 s  231353449 s          0 s
  Memory: 32.0 GB (30233.2890625 MB free)
  Uptime: 617612.74 sec
  Load Avg:  0.61  0.31  0.29
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, skylake-avx512)
  Threads: 1 on 40 virtual cores
Environment:
  XDG_CACHE_HOME = /home/jovyan/.cache/
  HOME = /home/jovyan
  PATH = /opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/jovyan/.local/bin
  TERM = xterm-color


[ Info: Julia executable: /home/jovyan/gbdt_environment_test-master/julia-1.9.2/bin/julia
[ Info: Trying to import PyCall...
┌ Error: `import PyCall` failed
│   exception =
│    ArgumentError: Package PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0] is required but does not seem to be installed:
│     - Run `Pkg.instantiate()` to install all recorded dependencies.
│    
│    Stacktrace:
│     [1] _require(pkg::Base.PkgId, env::Nothing)
│       @ Base ./loading.jl:1774
│     [2] _require_prelocked(uuidkey::Base.PkgId, env::Nothing)
│       @ Base ./loading.jl:1660
│     [3] _require_prelocked
│       @ ./loading.jl:1658 [inlined]
│     [4] macro expansion
│       @ ./lock.jl:267 [inlined]
│     [5] require(uuidkey::Base.PkgId)
│       @ Base ./loading.jl:1655
│     [6] top-level scope
│       @ ~/.local/lib/python3.9/site-packages/julia/install.jl:36
│     [7] include(mod::Module, _path::String)
│       @ Base ./Base.jl:457
│     [8] exec_options(opts::Base.JLOptions)
│       @ Base ./clie

## import python packages

In [2]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from tqdm import trange, tqdm
from itertools import product, chain
from scipy.stats.qmc import LatinHypercube as LHS

import seaborn as sns
sns.set()

from scipy.stats import norm, invgamma, rv_discrete
from scipy import integrate
from sklearn.neighbors import NearestCentroid
from sklearn.preprocessing import OrdinalEncoder
from scipy.stats import ttest_ind, chisquare
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize_scalar, minimize
import pickle
from IPython.display import display

## set up julia runtime

In [3]:
%%time
import julia, os

# julia_path_ = '/home/jungadam/idms/julia-1.9.2/bin/julia' 
julia_path_ = 'julia-1.9.2/bin/julia'

julia.Julia(runtime=julia_path_, compiled_modules=False)

%load_ext julia.magic

Initializing Julia interpreter. This may take some time...
CPU times: user 24.4 s, sys: 1.5 s, total: 25.9 s
Wall time: 25.2 s


## install & import julia packages

In [4]:
%%julia
using Pkg
Pkg.add(["DataFrames", "StatsBase", "Integrals", "Distributions",
        "Statistics", "Optim", "ProgressMeter"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Adapt ─────────────────────── v3.6.2
   Installed GPUArraysCore ─────────────── v0.1.5
   Installed Calculus ──────────────────── v0.5.1
   Installed IrrationalConstants ───────── v0.2.2
   Installed Crayons ───────────────────── v4.1.1
   Installed DiffRules ─────────────────── v1.15.1
   Installed FunctionWrappersWrappers ──── v0.1.3
   Installed Tricks ────────────────────── v0.1.7
   Installed Rmath ─────────────────────── v0.7.1
   Installed DualNumbers ───────────────── v0.6.8
   Installed HCubature ─────────────────── v1.5.1
   Installed TableTraits ───────────────── v1.0.1
   Installed StatsFuns ─────────────────── v1.3.0
   Installed HypergeometricFunctions ───── v0.3.23
   Installed SciMLBase ─────────────────── v1.94.0
   Installed ADTypes ───────────────────── v0.1.6
   Installed PositiveFactorizations ────── v0.2.4
   Installed SentinelArrays ────────────── v1.4.0
   I

In [5]:
%%julia
using Optim, ProgressMeter
include("Julia_CRPS_Scoring.jl")
using .Julia_CRPS_Scoring

## test run with a selection of models

### model implementations

In [6]:
from scipy.stats import ecdf as ecdf_special
class ECDF_RV:
    """wrapper for work with ecdf result as standard scipy RVs"""
    
    def __init__(self, ecdf_rv, samples):
        self.ecdf_rv = ecdf_rv
        self.samples = samples
        
    def cdf(self, x):
        return self.ecdf_rv.cdf.evaluate(x)
    
    def ppf(self, x):
        i = np.searchsorted(self.ecdf_rv.cdf.probabilities, x)
        return self.ecdf_rv.cdf.quantiles[i]
    
    def mean(self):
        return self.samples.mean()
    
    def var(self):
        return self.samples.var()
    
ecdf = lambda samples : ECDF_RV(ecdf_special(samples), samples)

In [7]:
def fit_MM_inv_gamma(m, s):
    """
    fit inv gamma to first two moments 'm' and 's',
    with method of moments.
    """
    
    # assert s > 0, "elfajult szoras"
    if s == 0:
        return rv_discrete(name='custm', values=(m, 1))
        
    
    fit_alpha = m**2 / s + 2
    fit_scale = m * (fit_alpha - 1)
    
    rv = invgamma(a = fit_alpha, scale = fit_scale, loc = 0)
    
    if np.isnan(rv.mean()) or np.isnan(rv.var()):
        print(m, s,  fit_alpha, fit_scale)
        raise Exception("b fit")
    
    return rv

fit_MM_inv_gamma = np.vectorize(fit_MM_inv_gamma)

In [8]:
def auto_scale_variance(loc, var, y):
    """return optimal_variance_coeff, CRPS_score"""
    def f(s, loc = loc, y = y):
        var_scaled = var * s
        var_scaled[var_scaled < 0] = 0
        crps = %julia CRPS_distribution($loc, $var_scaled, $y)
        return crps.mean()

    ubound = 1e15
    res = minimize_scalar(f, bounds = [0, ubound])
    if np.isclose(ubound, res.x):
        raise Exception("optim error")
        
    return res.x, res.fun

In [9]:
class UncertaintyMeasure:
    def __init__(self, model_params = {}):
        self.model = lgb.LGBMRegressor(**model_params)
        
    def fit(self, X_train, y_train, X_opt, y_opt, model_fit_params = {}):
        self.X_train = X_train
        self.y_train = y_train
        self.model.fit(self.X_train, 
                       np.log(self.y_train), **model_fit_params)
        return self
        
    def predict_loc(self, X_pred):
        return np.exp(self.model.predict(X_pred))

    def predict_rv(self, X):
        """pont becsles"""
        y = np.exp(self.model.predict(X))
        return np.array([rv_discrete(values=(m, 1)) for m in y])

    
    def crps_score(self, X_val, y_val):
        loc = self.predict_loc(X_val)
        samples = loc.reshape(-1,1)
        crps_score = %julia CRPS_ecdf($samples, $y_val)   
        return crps_score
    
    
    def skill_scores(self, X_val, y_val, ref_scores):
        crps_score = self.crps_score(X_val, y_val)
        skill_scores = {"crps" : crps_score, 
                          **{n : skill_score(crps_score, ref_scores[n].mean()) 
                                                 for n in ref_scores.columns[:-1]}}
        return pd.DataFrame.from_dict(skill_scores)

    
    def score(self, X_val, y_val, ref_scores):
        """
        retrun skill scores and p value
        on validation set
        """
        skill_scores = self.skill_scores(X_val, y_val, ref_scores)
        pvalue = ttest_ind(ref_scores["location_ig"], skill_scores["crps"]).pvalue
        return {**skill_scores.mean().to_dict(), "p_value" : pvalue}

In [10]:
class UMMoments(UncertaintyMeasure):
    """
    uncertainty measure using a 2 parameter
    distribution (e.g. inverse gamma)
    """
    
    def __init__(self, model_params = {}, distribution = "InverseGamma"):
        super().__init__(model_params)
        self.distribution = distribution
        self.affin_coeffs = (1,0) # (a,b) : var := a * var + b
        
    def get_moments(self, X_pred):
        """point predict"""
        loc = self.predict_loc(X_pred)
        var = np.zeros_like(loc)
        return loc, var
        
    def predict_rv(self, X):
        loc, var = self.get_moments(X)
        
        assert self.distribution == "InverseGamma", f"{self.distribution} is not implemented"
        return fit_MM_inv_gamma(loc, var)
        
    def crps_score(self, X_val, y_val):
        loc, var = self.get_moments(X_val)
        var[var<0] = 0
        distr = self.distribution
        crps_score = %julia CRPS_distribution($loc, $var, $y_val, $distr)
        return crps_score
    
    
    def auto_tune_affin_global(self, X_opt, y_opt):
        """find optimal variance scaling on `opt` dataset"""
        
        loc, var = self.get_moments(X_opt)
        a, crps = auto_scale_variance(loc, var, y_opt)
        
        self.affin_coeffs = (a,0)
        
        return crps

In [11]:
class Zero_var(UMMoments):
    pass

In [12]:
class Optimal_var(UMMoments):  
    
    def optimal_crps(self, X_val, y_val):
        loc = self.predict_loc(X_val)
        y_val = y_val.to_numpy()
        %julia loc = $loc
        %julia obs = $y_val

        #maximal observed squared error
        max_var = (loc - y_val).max()**2
        %julia max_var = $max_var

        julia.Main.eval("""
            mins = []
            minimizers = []
            @showprogress "search optimal variances for prediction set" for (l,o) in zip(loc, obs)
                f(s) = CRPS_distribution([l], [s], [o])[1]
                res = optimize(f, 0, max_var) 
                push!(minimizers, res.minimizer)
                push!(mins, res.minimum)
            end
                        """)
        opt_crps = %julia mins
        opt_vars = %julia minimizers

        return opt_crps, opt_vars
    
    def crps_score(self, X_val, y_val):
        crps, _ = self.optimal_crps(X_val, y_val) 
        return crps
        

In [13]:
class LocAffin_var(UMMoments):
    """
    loc = gbdt prediction
    var = a * loc + b
    """
    
    def fit(self, X_train, y_train, X_opt, y_opt):
        super().fit(X_train, y_train, X_opt, y_opt)
        self.optimize_var_coeffs(X_opt, y_opt)  
        return self
    
    def get_moments(self, X_pred):
        loc = np.exp(self.model.predict(X_pred))
        var = loc * self.var_coeffs[0] + self.var_coeffs[1]
        var[var <= 0] = 0
        return loc, var
    
    def optimize_var_coeffs(self, X_opt, y_opt):
        loc = np.exp(self.model.predict(X_opt))
        def f(x, y_opt = y_opt):
            var = loc * x[0] + x[1]
            var[var <= 0] = 0
            crps = %julia CRPS_distribution($loc, $var, $y_opt)
            return crps.mean()

        res = minimize(f, x0 = [1,0])
        self.var_coeffs = res.x

In [14]:
class Constant_var(UMMoments):
    """
    loc = gbdt prediction
    var = const
    """

    def fit(self, X_train, y_train, X_opt, y_opt):
        super().fit(X_train, y_train, X_opt, y_opt)
        self.optimize_var_coeff(X_opt, y_opt)  
        return self
    
    def get_moments(self, X_pred):
        loc = np.exp(self.model.predict(X_pred))
        var = np.ones_like(loc) * self.var_coeff
        var[var <= 0] = 0
        return loc, var
    
    def optimize_var_coeff(self, X_opt, y_opt):
        loc = np.exp(self.model.predict(X_opt))
        def f(x, y_opt = y_opt):
            var = np.ones_like(loc) * x
            var[var <= 0] = 0
            crps = %julia CRPS_distribution($loc, $var, $y_opt)
            return crps.mean()

        res = minimize_scalar(f)
        self.var_coeff = res.x

In [15]:
class InstanceBasedUncertainty(UMMoments):
    def fit(self,X_train, y_train, X_opt, y_opt, k = 40, model_fit_params = {}):
        super().fit(X_train, y_train, X_opt, y_opt)
        
        self.L_train = self.model.predict(X_train, pred_leaf = True)
        self.k = k
        
        self.auto_tune_affin_global(X_opt, y_opt)
        self.auto_tune_k(X_opt, y_opt)
        return self
    
    def get_y_simmilarities(self, X_pred):
        L_pred = self.model.predict(X_pred, pred_leaf = True)
        L_train_b = self.L_train.T[np.newaxis, :,:]
        L_pred_b = np.broadcast_to(L_pred[:,:,np.newaxis], 
                                   shape = (*L_pred.shape, self.L_train.shape[0]))
        leaf_matches = L_train_b == L_pred_b
        
        sum_of_same_leaf = leaf_matches.sum(axis = 1)
        y_train_br = np.broadcast_to(self.y_train.values[np.newaxis, :], shape = sum_of_same_leaf.shape)
        perm = np.argsort(sum_of_same_leaf, axis = 1)
        simmilar_train_y = np.take_along_axis(y_train_br, perm, axis = 1)
        return simmilar_train_y
    
    def get_moments(self, X_pred):
        loc =  np.exp(self.model.predict(X_pred))
        

        simmilar_train_y = self.get_y_simmilarities(X_pred)
        k = self.k
        var = simmilar_train_y[:,-k:].var(axis = 1)
        var = self.affin_coeffs[0] * var + self.affin_coeffs[1]
        return loc, var
    
    def auto_tune_k(self, X_opt, y_opt):
        simmilar_train_y = self.get_y_simmilarities(X_opt)
        loc =  np.exp(self.model.predict(X_opt))

        def f(k, y_opt = y_opt):
            if k <= 0:
                return np.inf
            k = int(k)
            
            var = simmilar_train_y[:,-k:].var(axis = 1)
            var = self.affin_coeffs[0] * var + self.affin_coeffs[1]
            var[var<0]= 0
            lloc, ly_opt = loc, y_opt
            crps = %julia CRPS_distribution($lloc, $var, $ly_opt)
            return crps.mean()
        

        test_k = [*range(2,100)] #[3,4,5,10,20,30,40,60,80,100]
        test = [(k, f(k)) for k in tqdm(test_k, desc= "optimise k", leave=False)]
        
        k, m = min(test, key = lambda t : t[1])
        self.k = k
        
        
        return k, m, test
            

### test models

In [16]:
#load sgem matrix product dataset
df = pd.read_csv("sgemm_product.csv")

runCols = ["Run1 (ms)", "Run2 (ms)", "Run3 (ms)", "Run4 (ms)"]
df["minRun"] = df[runCols].min(axis = 1)
df["meanRun"] = df[runCols].mean(axis = 1)

X = df.drop(columns = [*runCols, "minRun", "meanRun"])
target = "minRun"
Y = df[target]

In [17]:
#split data 

# validate model:
X_data, X_val, y_data, y_val = train_test_split(X, Y, test_size = 5000, random_state=1)
# fine tune hyperparam:
X_data, X_opt, y_data, y_opt = train_test_split(X_data, y_data, test_size = 999, random_state=1)
# train model:
X_train, X_data, y_train, y_data = train_test_split(X_data, y_data, train_size = 1000, random_state=1)

In [18]:
train_set = X_train, y_train, X_opt, y_opt

In [19]:
models = [Zero_var, LocAffin_var, Constant_var, InstanceBasedUncertainty, Optimal_var]

In [20]:
scores = {}
for Model in models:
    print(Model.__name__, "\n")
    m = Model().fit(X_train, y_train, X_opt, y_opt)
    crps = m.crps_score(X_val, y_val)
    scores[Model.__name__] = crps

Zero_var 



NameError: name 'loc' is not defined

In [ ]:
scores_df = pd.DataFrame(scores)
results = pd.DataFrame(scores_df.mean(), columns = ["CRPS"])

In [ ]:
r = scores_df["Constant_var"].mean()
o = scores_df["Optimal_var"].mean()
results["skill_score"] = [((r - scores_df[c]) / (r - o)).mean() for c in scores_df.columns]
results["skill_score"] = results["skill_score"].round(3)

In [ ]:
results.sort_values(by = "CRPS")